# Standings Test
First we establish a valid oauth token below:

In [45]:
from yahoo_oauth import OAuth2
oauth = OAuth2(None, None, from_file='../oauth2.json')

[2019-08-07 22:04:51,587 DEBUG] [yahoo_oauth.yahoo_oauth.__init__] Checking 
[2019-08-07 22:04:51,589 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 4039.62246966362
[2019-08-07 22:04:51,590 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN HAS EXPIRED
[2019-08-07 22:04:51,591 DEBUG] [yahoo_oauth.yahoo_oauth.refresh_access_token] REFRESHING TOKEN


This will run more frequently - this will check if the token is valid, if not, refresh. Then, we can try to download info from yahoo.

In [48]:
from bs4 import BeautifulSoup
import xml.etree.ElementTree as et 
import pandas as pd 



if not oauth.token_is_valid():
    oauth.refresh_access_token()
    
df_cols = ["team name", "owner name", "wins", "losses"]
out_df = pd.DataFrame(columns = df_cols)

num_teams = 10

    
url = "https://fantasysports.yahooapis.com/fantasy/v2/league/nfl.l.854917/standings"
# probably migrate to scoreboard when there is data... but unsure
# url = "https://fantasysports.yahooapis.com/fantasy/v2/league/nfl.l.854917/scoreboard"
payload = ""
response = oauth.session.get(url, params=payload)

# we only want to process data if there's data to pass.
if not response.status_code == 200:
    print("Status Code Not Valid")
    print(response.text)
else:

#   XML Samples which are searchable can be found here: https://developer.yahoo.com/fantasysports/guide/

    soup = BeautifulSoup(response.text, 'lxml')
#     Remove new line formatting from html to prevent confusion
    clean = str(soup).replace('\n', '')
#     print(clean)
#     print(soup.prettify())

#  Get team Names:
#     select all name tags beneath team tags
    s_team_names = soup.select('team name')
#     extract the text from the entire tag
    s_team_names = list(map(lambda x: x.text, s_team_names))
#     place that in a fancy little pandas series
    s_team_names = pd.Series(s_team_names)
    
#  Get owner name:
#     select all wins tags beneath team tags
    s_nickname = soup.select('team nickname')
#     extract the text from the entire tag
    s_nickname = list(map(lambda x: x.text, s_nickname))
#     place that in a fancy little pandas series
    s_nickname = pd.Series(s_nickname)
    
#  Get team wins:
#     select all wins tags beneath team tags
    s_wins = soup.select('team outcome_totals wins')
#     extract the text from the entire tag
    s_wins = list(map(lambda x: x.text, s_wins))
#     place that in a fancy little pandas series
    s_wins = pd.Series(s_wins)

#  Get team losses:
#     select all losses tags beneath team tags
    s_losses = soup.select('team losses')
#     extract the text from the entire tag
    s_losses = list(map(lambda x: x.text, s_losses))
#     place that in a fancy little pandas series
    s_losses = pd.Series(s_losses)
    
out_df['team name'] = s_team_names
out_df['owner name'] = s_nickname
out_df['wins'] = s_wins
out_df['losses'] = s_losses
    
display(out_df)

    #     Documentation that's helpful for ElementTree: https://docs.python.org/2/library/xml.etree.elementtree.html#finding-interesting-elements
    # to traverse the tree we need to implement some namespacing: https://stackoverflow.com/questions/36368469/elementtree-find-always-returns-none?rq=1
    # Too much effort; using beautiful soup
    #     xtree = et.fromstring(clean)
    #     s_name = xtree.find("./*/*/*/name")
    #     print(s_name)
    

[2019-08-07 22:07:20,834 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 149.24266600608826
[2019-08-07 22:07:20,835 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID


,team name,owner name,wins,losses
0,Commissar Nofunski,asher,0,0
1,FranzFundinand,alexanderhill96,0,0
2,3rd Time’s the Charm,Alex,0,0
3,ITouchedTucker,Jessica,0,0
4,The OG Champ,morgan0717,0,0
5,WineMan,Adam,0,0
6,Tusken Raiders,Ben,0,0
7,GoldenReceivers,Gabe,0,0
8,Who needs Buffalo ?,MARTIN,0,0
9,NotMyUsualTeamName,Dave French,0,0
